## 6.4 – Functional Programming Concepts
### Functional Programming
We have taken a short surface-level journey through the history of programming paradigms, in a *very* loose chronological order of when they were invented or popular. First we have *scripts*, code with little structure at all. Then we have *procedural programming*, where *data structures* are used and passed between *procedures* to manipulate them. Then we talked about *object-oriented programming*, where *data* and *functionality* are combined into *objects*.

**Functional programming** is somewhat tangential to the other paradigms. It has been around a long time, as long as the early procedural languages. In fact, you might argue that functional programming has grown along an entirely separate, parallel branch of computer science. Algorithms today are normally designed around a certain sequential manipulation of data that was formally defined by the [Turing machine](https://en.wikipedia.org/wiki/Turing_machine). The Turing machine itself was invented by Alan Turing to solve [a mathematical problem](https://en.wikipedia.org/wiki/Entscheidungsproblem), but he was actually beaten to this goal by a few months by Alonzo Church, who used his own description of an algorithm: [the lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus). The lambda calculus is the origin of functional programming.

It has a long history as a somewhat “alternative” to other programming paradigms. But functional programming concepts are increasingly becoming popular in non-functional languages, like Python.

### Functions as Computation
We are already familiar with the concept of functions. Do you remember the difference between a function and a procedure? A function always returns a value, and procedure does not. If a procedure does not return a value, that means it must be able to *do something else*. It might change the value of some object that is passed in, or it might interact with the “outside world” somehow: it might print something to the screen, connect to the Internet, or turn on your printer. All of the things a procedure can do that *aren't* returning a value are called **side-effects**.

Functional programming is concerned with **pure functions**. Functions that do not have side-effects, and are also not influenced by the external state in any way: the function always behaves the same way for a given input.

The interesting thing, and the connection with the lambda calculus, is that it is possible to write any Turing-complete program (algorithm) by using only pure functions and recursion. Literally! A programming language does not need to provide any other features: numbers, if statements, equality tests... none of it. Just functions, and the ability to apply functions to other functions.

Functional languages are not *this* stark. They do provide the nice features we expect. Good functional code uses a lot of lists, function application, and recursion (specifically tail recursion: a concept not found in Python but is interesting to read up on).

That's all well and good, but let's look at what this really means for Python development.

### “Functional” Python
#### Functions as Variables
Here is something fun...

In [1]:
my_fun = len
my_fun([0, 1, 2])

3

How about that? A small number of you may have discovered something like this in the past, and are wondering what the big deal is. Everyone else: you may pick up your jaw off the ground.

Yes, it is possible to assign functions to variables, and then those variables act like functions. In Python, functions are objects too. When you `def` a function, you give it a name, but you can also assign it to another one. Both names then point at the same function object.

Functions can be added to data structures, like lists or dictionaries:

In [8]:
def round_down(num):
    return int(num // 1)

def round_up(num):
    if num == round_down(num):
        return round_down(num)
    else:
        return round_down(num) + 1

# This dictionary contains three function objects
three_rounds = [round_down, round_up, round]

x = 9.5
for f in three_rounds:
    print(f(x))

9
10
10


#### Functions as Arguments
Functions can also be passed as arguments into other functions. This is where stuff starts looking really *functional*.

In [11]:
def apply_to_list(function, in_list):
    out_list = []
    for item in in_list:
        out_list.append(function(item))
    return out_list

def square(x):
    return x**2

my_list = [1, 2, 3, 4, 5]

print(apply_to_list(square, my_list))

[1, 4, 9, 16, 25]


The function above `apply_to_list` takes a function and a list then returns the list generated by applying the function to each element of that list. This is a very common thing to want to do in a functional language, and the function has a special name: `map`. 

Python has a `map` function builtin. It returns a generator-like object, like the function `range` – we saw this behaviour before in [Section 4.3](..Chapter%204/4.3.ipynb#Dictionary-Comprehensions).

In [14]:
map(square, my_list)

We can inspect the values of the `map` object using a normal for-each loop:

In [10]:
for item in map(square, my_list):
    print(item)

1
2
3
4
5


Again the way the map works allows it to use *lazy evaluation*. The function is not actually called on the list elements until it is needed. We can demonstrate this if we introduce some side-effects into the function:

In [16]:
def square(x):
    print(f"Okay, I will square {x}!")
    return x**2

my_map = map(square, [1, 2, 3, 4, 5])
my_map

The code above has created the map object, but has not actually run the function on the lists elements. If it had, we would have seen a bunch of print statements.

Let's try inspecting the values of the map object again. Now the function will actually be called, one element at a time, so we can use the result in our for-each loop:

In [17]:
for item in my_map:
    print(item)

Okay, I will square 1!
1
Okay, I will square 2!
4
Okay, I will square 3!
9
Okay, I will square 4!
16
Okay, I will square 5!
25


It's worth pointing out here that we should not forget *list comprehensions* (also covered in [Section 4.3](../Chapter%204/4.3.ipynb)). In this specific situation we would probably actually be better off writing:

In [18]:
my_list = [1, 2, 3, 4, 5]
squares = [x**2 for x in my_list]
squares

[1, 4, 9, 16, 25]

#### Functions as Return Values
We can define functions inside functions, and we can return those functions from the outer functions!

You can think of this as writing a function that *generates functions*:

In [22]:
def adder(x):
    def add_x(num):
        return num + x
    return add_x

add_2 = adder(2)
add_2(5)

7

As with most syntax in Python, we can also chain this to do both things at once:

In [23]:
result = adder(5)(10)
result

15

Although this is unlikely to be the intended use of anyone writing the original function. If it were, they could create a function that accepted two parameters.

Here is another example. Suppose we wanted to supply a function that converted from GBP to USD. We want to always use the most up-to-date value for the currency conversion rate when the function is created – maybe we use some kind of online API to find out that value. But from then on the conversion rate will not change (unless the user generates the function again).

In [30]:
# this function fills-in for an API call
def get_rate():
    return 1.25

def get_gbp_usd_function():
    gbp_to_usd = get_rate()
    def convert(num):
        return num * gbp_to_usd
    return convert
    
convert_gbp_usd = get_gbp_usd_function()
pounds = 100
dollars = convert_gbp_usd(pounds)
print(f"£{pounds} is ${dollars}")

£100 is $125.0


The *inner function* in this case looks like it is called `convert`, but we cannot actually access that function outside of the *scope* of the *outer function* `get_gbp_usd_function`.

#### Lambda Expression
In this situation, we do not even need to give the inner function a name. We can use a **lambda expression**. This is a simple one line method for creating an **anonymous function**, one which has no name attached. Here is a simple example:

In [32]:
multiply = lambda x, y: x * y
multiply(5, 10)

50

The syntax for the lambda expression is the word `lambda`, followed by the parameters, followed by a colon `:`, followed by the value to return.

You should not normally assign a lambda expression to a variable. If we are going to give the function a name, we should just write a `def`. But we can use the syntax to tidy up our previous mapping operation:

In [33]:
my_map = map(lambda x: x**2, [1, 2, 3, 4, 5])

for item in my_map:
    print(item)

1
4
9
16
25


Though this is still not as elegant as using a list comprehension in this specific example.

The lambda expression *does* however improve our currency conversion example:

In [31]:
# this function fills-in for an API call
def get_rate():
    return 1.25

def get_gbp_usd_function():
    gbp_to_usd = get_rate()
    return lambda x: x * gbp_to_usd
    
convert_gbp_usd = get_gbp_usd_function()
pounds = 100
dollars = convert_gbp_usd(pounds)
print(f"£{pounds} is ${dollars}")

£100 is $125.0


#### Lambdas as Arguments
We showed above that we could use a lambda expression as an argument for a `map`. It's a very direct application of the lambda expression, and in most cases a list comprehension would be better.

Some other builtin functions also take functions as parameters where a lambda expression is perfect.

Consider the `max` function. When we get a maximum value from a list of numbers, we pretty much know what to expect. But not all objects can be compared so obviously.

Suppose we are writing a program which models the final grades of a set of students. Each student has a name and a dictionary which maps from subjects to grades. In the code below I've written the class, followed by some code which generates some objects and puts them in a list.

In [65]:
class Student:
    def __init__(self, name = "", grades = {}):
        self.name = name
        self.grades = grades
        
students = []
students.append(Student("Riley", {"english": 30, "maths": 58, "computer science": 34, "history": 6}))
students.append(Student("Mason", {"english": 51, "maths": 50, "computer science": 65, "history": 87}))
students.append(Student("Lindsay", {"english": 6, "maths": 5, "computer science": 4, "history": 44}))
students.append(Student("Logan", {"english": 98, "maths": 21, "computer science": 85, "history": 33}))
students.append(Student("Ashley", {"english": 69, "maths": 68, "computer science": 92, "history": 82}))

Now if we query this list to find the maximum element, what should the correct result be?

In [68]:
max(students)

TypeError: '>' not supported between instances of 'Student' and 'Student'

In fact, we cannot, because the `Student` class does not allow us to make comparisons. We can write our own `__lt__` (less than) and `__gt__` (greater than) methods, and then `max` would return the object which gave the maximum result by this definition of `x > y`. However this isn't what we're really looking for either. We might want to find the student that has the maximum English mark, or the maximum history mark.

Maximum has an optional named parameter called `key`:

In [70]:
help(max)

Help on built-in function max in module builtins:

max(...)
    max(iterable, *[, default=obj, key=func]) -> value
    max(arg1, arg2, *args, *[, key=func]) -> value
    
    With a single iterable argument, return its biggest item. The
    default keyword-only argument specifies an object to return if
    the provided iterable is empty.
    With two or more arguments, return the largest argument.



We can write a lambda expression to tell the `max` function how it should interpret each object in the list. It will run the function on each object, and then use the result to perform the comparison. So, if we want to know the student with the highest computer science mark, we can write a function which finds the right key in the dictionary:

In [72]:
best_student = max(students, key = lambda s: s.grades["computer science"])
print(f"The best student is {best_student.name} with a CS grade of {best_student.grades['computer science']}")

The best student is Ashley with a CS grade of 92


Try modifying the code above to use a lambda expression to find the student who has the highest combined mark across all their grades. Can you make your code work even if new subjects were later added to the dictionary?

### What's Next
We've only scratched the surface of functional programming, even limiting ourselves to its applications in Python. If you really want to get into it, go and look up some tutorials for a functional language like Haskell, there is even one embedded on the [Haskell website](https://www.haskell.org/).

But for this unit, there is just one thing left, which is a final video which puts all of these concepts together. To do that, we are going to look at the code that generated the quizzes you took in the first few chapters, which was all written in Python. [Click here to go to the final section](6.5.ipynb).